In [1]:
%%capture
!pip install -r ../model-archiver/model-store/youtubegoes5g/requirements.txt

In [2]:
from torch import nn
import torch

# Build model with non-linear activation function
class InterruptionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer_1 = nn.Linear(in_features=29, out_features=200)
        self.layer_2 = nn.Linear(in_features=200, out_features=100)
        self.layer_3 = nn.Linear(in_features=100, out_features=1)
        self.relu = nn.ReLU() # <- add in ReLU activation function
        # Can also put sigmoid in the model
        # This would mean you don't need to use it on the predictions
        # self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # Intersperse the ReLU activation function between layers
        return self.layer_3(self.relu(self.layer_2(self.relu(self.layer_1(x)))))

In [3]:
import os
from pickle import load
from sklearn.preprocessing import StandardScaler

model = None

class MyHandler():
    def __init__(self, model):
        self.model = model

    def preprocess(self, data):
        """
        Transform raw input into model input data.
        """
        try:
            # Load scaler
            scaler = StandardScaler()
            scaler = load(open('../model-archiver/model-store/youtubegoes5g/scaler.pkl', 'rb'))

            tensor_list = []
            for item in data:
                item = scaler.transform([item['data']])
                tensor_data = torch.tensor(item, dtype=torch.float32)  # Each instance as a tensor
                tensor_list.append(tensor_data)
            # Stack all tensors along a new dimension to create a single tensor
            combined_tensor = torch.cat(tensor_list, dim=0)
            return combined_tensor
        except Exception as e:
            raise ValueError("Failed to preprocess input data: ", e, "data received: ", data)

    def inference(self, model_input):
        """
        Perform model inference.
        """
        try:
            inference_list = []
            for tensor_data in model_input:
                with torch.no_grad():
                    output = torch.round(torch.sigmoid(self.model(tensor_data))).squeeze()
                #inference = output.cpu().numpy().tolist()
                inference_list.append(output)
            return inference_list
        except Exception as e:
            raise RuntimeError("Inference failed:", e)

    def postprocess(self, inference_output):
        """
        Convert model output to a list of predictions.
        """
        try:
            # Process each item in the batch
            result_list = []
            for result in inference_output:
                if result > 0:
                    result_list.append("Stall")
                else:
                    result_list.append("No Stall")
            return result_list
        except Exception as e:
            raise ValueError("Failed to postprocess output data: ", e)

    def handle(self, data):
        """
        Handle a prediction request.
        """
        try:
            model_input = self.preprocess(data)
            model_output = self.inference(model_input)
            return self.postprocess(model_output)
        except Exception as e:
            return [str(e)]

In [4]:
# Instantiate model and handler
serialized_file = "../model-archiver/model-store/youtubegoes5g/model.pt"
    
model = InterruptionModel()
model.load_state_dict(torch.load(serialized_file, weights_only=True))
model.eval()
handler = MyHandler(model)

/opt/conda/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [9]:
%%timeit -r 1 -n 1
# Single input test
data = [{"data": [13,13,13,0,13,13,13,13,-76,-76,-81,-76,-78.5,-76,-76,-7,-7,-12,-7,-9.5,-7,-7,12,12,7,12,9.5,12,12]}]

result = handler.handle(data)

print("result: " ,result)

result:  ['No Stall']
2.64 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [10]:
%%timeit -r 1 -n 1
data = [
      {"data": [13,13,13,0,13,13,13,13,-76,-76,-81,-76,-78.5,-76,-76,-7,-7,-12,-7,-9.5,-7,-7,12,12,7,12,9.5,12,12]},
      {"data": [13,13,12,0.4714045208,13,12.5,13,13,-81,-81,-82,-81,-81.5,-81,-81,-12,-12,-11,-12,-12,-12,-11.5,7,7,2,7,4.5,7,7]},
      {"data": [6,7,7,0.4714045208,7,6.5,7,7,-107,-106,-106,-106,-106.5,-106,-106,-13,-14,-14,-14,-14,-14,-13.5,2,2,2,2,2,2,2]},
      {"data": [8,8,8,0,8,8,8,8,-108,-108,-108,-108,-108,-108,-108,-13,-13,-13,-13,-13,-13,-13,2,2,2,2,2,2,2]}
]

result = handler.handle(data)

print("result: ", result)

result:  ['No Stall', 'No Stall', 'Stall', 'Stall']
3.46 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [11]:
%%timeit -r 1 -n 1

data1 = [{"data": [13,13,13,0,13,13,13,13,-76,-76,-81,-76,-78.5,-76,-76,-7,-7,-12,-7,-9.5,-7,-7,12,12,7,12,9.5,12,12]}]
data2 = [{"data": [13,13,12,0.4714045208,13,12.5,13,13,-81,-81,-82,-81,-81.5,-81,-81,-12,-12,-11,-12,-12,-12,-11.5,7,7,2,7,4.5,7,7]}]
data3 = [{"data": [6,7,7,0.4714045208,7,6.5,7,7,-107,-106,-106,-106,-106.5,-106,-106,-13,-14,-14,-14,-14,-14,-13.5,2,2,2,2,2,2,2]}]
data4 = [{"data": [8,8,8,0,8,8,8,8,-108,-108,-108,-108,-108,-108,-108,-13,-13,-13,-13,-13,-13,-13,2,2,2,2,2,2,2]}]

result1 = handler.handle(data1)
print("result 1:", result1)

result2 = handler.handle(data2)
print("result 2:", result2)

result3 = handler.handle(data3)
print("result 3:", result3)

result4 = handler.handle(data4)
print("result 4:", result4)

result 1: ['No Stall']
result 2: ['No Stall']
result 3: ['Stall']
result 4: ['Stall']
4.38 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [12]:
%%timeit -r 1 -n 1
from concurrent.futures import ThreadPoolExecutor

# Define a task function to handle data
def handle_data(data, identifier):
    handler = MyHandler(model)
    result = handler.handle(data)
    return f"Task {identifier} finished with result: {result}"


# Example datasets
datasets = [
    ([{"data": [13, 13, 13, 0, 13, 13, 13, 13, -76, -76, -81, -76, -78.5, -76, -76, -7, -7, -12, -7, -9.5, -7, -7, 12, 12, 7, 12, 9.5, 12, 12]}], "Data1"),
    ([{"data": [13, 13, 12, 0.4714045208, 13, 12.5, 13, 13, -81, -81, -82, -81, -81.5, -81, -81, -12, -12, -11, -12, -12, -12, -11.5, 7, 7, 2, 7, 4.5, 7, 7]}], "Data2"),
    ([{"data": [6, 7, 7, 0.4714045208, 7, 6.5, 7, 7, -107, -106, -106, -106, -106.5, -106, -106, -13, -14, -14, -14, -14, -14, -13.5, 2, 2, 2, 2, 2, 2, 2]}], "Data3"),
    ([{"data": [8, 8, 8, 0, 8, 8, 8, 8, -108, -108, -108, -108, -108, -108, -108, -13, -13, -13, -13, -13, -13, -13, 2, 2, 2, 2, 2, 2, 2]}], "Data4"),
]

# Run tasks concurrently
with ThreadPoolExecutor() as executor:
    futures = [executor.submit(handle_data, dataset[0], dataset[1]) for dataset in datasets]

    # Collect and print results as they complete
    for future in futures:
        print(future.result())

Task Data1 finished with result: ['No Stall']
Task Data2 finished with result: ['No Stall']
Task Data3 finished with result: ['Stall']
Task Data4 finished with result: ['Stall']
14.1 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
